In [18]:
!python Feature_Extract.py aes128_table_ecb key_in

!python Feature_Extract.py AES_PPRM1 Kin AES_PPRM1
!python Feature_Extract.py AES_PPRM3 Kin AES_PPRM3
!python Feature_Extract.py AES_TBL Kin AES_TBL
!python Feature_Extract.py RSA Kin RSA
!python Feature_Extract.py SABER pol_64bit_in SABER # change PLC's label to 0 manually
!python Feature_Extract.py PRESENT key PRESENT

/Users/ruixiang/PycharmProjects/SCAR/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Loading cached VCD object from: ../data/aes128_table_ecb/aes128_table_ecb_vcd.pkl
Loading toggle counts from cache: ../data/aes128_table_ecb/aes128_table_ecb_toggle.txt
Successfully loaded toggle counts from cache.
[INFO] Features written to ../out/features.csv
[INFO] Edges written to ../out/edges.csv
/Users/ruixiang/PycharmProjects/SCAR/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Loading cached VCD object from: ../data/AES_PPRM1/AES_PPRM1_vcd.pkl
Loading toggle counts from cache: ../data/AES_PPRM1/AES_P

In [158]:
!python SCAR_GNN.py

/Users/ruixiang/PycharmProjects/SCAR/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
num_features: 7, num_classes: 2
feature_names: ['Degree', 'Hamming distance', 'Paths', 'and', 'mux', 'or', 'xor']
Train label distribution:
label
0    161
1     66
Name: count, dtype: int64

Test label distribution:
label
0    161
1     66
Name: count, dtype: int64
Epoch 1/32
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - acc: 0.3335 - loss: 10.1911 - precision: 0.2835 - recall: 0.9000 - val_acc: 0.4783 - val_loss: 3.3823 - val_precision: 0.1818 - val_recall: 0.4000
Epoch 2/32
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc: 0.3884 - loss: 9.0654 - precision: 0.3386 - recall: 0.9093 - val_acc: 0.6087 - val_loss: 3.8433 - val_precision: 0.2500 - val_recall: 0.4000
Epoch 3/32
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 

In [159]:
!python test.py

/Users/ruixiang/PycharmProjects/SCAR/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Found 6 dataset(s): ['AES_PPRM1', 'AES_PPRM3', 'AES_TBL', 'PRESENT', 'RSA', 'SABER']
Testing AES_PPRM1: read ../test/AES_PPRM1_features.csv and ../test/AES_PPRM1_edges.csv
Tensor("GatherV2:0", shape=(1, 32), dtype=float32)
Tensor("gnn_model_1/GatherV2:0", shape=(1, 32), dtype=float32)
Tensor("gnn_model_1/GatherV2:0", shape=(1, 32), dtype=float32)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 304ms/step
Tensor("gnn_model_1/GatherV2:0", shape=(None, 32), dtype=float32)
Testing AES_PPRM3: read ../test/AES_PPRM3_features.csv and ../test/AES_PPRM3_edges.csv
Tensor("GatherV2:0", shape=(1, 32), dtype=float32)
Tensor("gnn_model_1/GatherV2:0", shape=(1, 32), dtype=float32)
Tensor("gnn_model_1/GatherV2:0", shape=(1, 32), dtype=float